In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Parameter dasar
image_size = (150, 150)  # Sesuaikan dengan ukuran gambar saat pelatihan

# Load model yang telah disimpan
model = tf.keras.models.load_model("document_classifier.h5")

# Fungsi untuk memuat class indices dari model sebelumnya
def load_class_indices(train_generator_path="./dataset"):
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
    datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        train_generator_path,
        target_size=image_size,
        batch_size=32,
        class_mode="categorical",
        subset="training"
    )
    return train_generator.class_indices

# Mendapatkan class_indices
class_indices = load_class_indices()
class_names = {v: k for k, v in class_indices.items()}  # Membalik indeks

# Fungsi untuk Klasifikasi Gambar Baru
def classify_image(image_path):
    # Load gambar
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prediksi
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)

    # Logika untuk klasifikasi "Other" jika confidence rendah
    if confidence < 0.8:  # Threshold dapat disesuaikan
        return "Other", confidence
    else:
        return class_names[predicted_class], confidence

# Contoh Penggunaan
image_path = "./testing/sim2.png"  # Path ke gambar uji
predicted_class, confidence = classify_image(image_path)
print(f"Kelas: {predicted_class}, Confidence: {confidence:.2f}")

Found 1323 images belonging to 4 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Kelas: KK, Confidence: 1.00
